In [51]:
## Importing libraries 
import nemo
import nemo.collections.asr as nemo_asr
import jiwer
from jiwer import wer
from jiwer import cer
import json
import zipfile
import os 
import librosa
import torch
import torch.nn as nn
import copy
import pytorch_lightning as pl
import time 
import torchaudio
import noisereduce as nr
from scipy.io import wavfile
from omegaconf import OmegaConf, open_dict
from nemo.collections.asr.models.ctc_bpe_models import EncDecCTCModelBPE
from nemo.core.config import hydra_runner
from nemo.utils import logging
from nemo.utils.exp_manager import exp_manager

In [84]:
# Initialise quartznet model 
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_quartznet15x5")

[NeMo I 2022-07-13 11:20:57 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.
[NeMo I 2022-07-13 11:20:57 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo
[NeMo I 2022-07-13 11:20:57 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-13 11:20:58 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: /asr_set_1.2/train/train_{0..1023}.tar
    num_workers: 20
    
[NeMo W 2022-07-13 11:20:58 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
   

[NeMo I 2022-07-13 11:20:58 features:200] PADDING: 16
[NeMo I 2022-07-13 11:20:59 save_restore_connector:243] Model EncDecCTCModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.


In [8]:
manifest_path = '/home/dgxuser/Desktop/kailin/manifest_kl.json'
script_path = '/home/dgxuser/Desktop/kailin/script_kl.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/audios_kl/'
n = 1 
with open(manifest_path, 'w') as fout:
    with open (script_path, 'r') as script:
        while (n <= 52): 
            text = script.readline().strip()
            file_dir = audio_dir + str(n) + '.wav'
            y, sr = torchaudio.load(file_dir)
            y = y.mean(dim=0) # if there are multiple channels, average them to single channel
            if sr != 16000:
                resampler = torchaudio.transforms.Resample(sr, 16000)
                y_resampled = resampler(y)
            file_dir = audio_dir + str(n) + '_resampled.wav'
            torchaudio.save(file_dir, y.unsqueeze(0), sr)
            duration = librosa.core.get_duration(filename=file_dir)
            metadata = {
            "audio_filepath": file_dir,
            "duration": duration, 
            "text": text
            }
            json.dump(metadata, fout)
            fout.write('\n')   
            n = n + 1 

In [53]:
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
error = wer(transcript, predicted)
print('Word Error Rate (Resampled): ' + str(error))

Word Error Rate (Resampled): 0.05419580419580419


In [54]:
error = cer(transcript, predicted)
print('Character Error Rate (Resampled): ' + str(error))

Character Error Rate (Resampled): 0.022117962466487937


In [20]:
for i in range(len(transcript)): 
    if (transcript[i] != predicted[i]): 
        print(transcript[i])
        print(predicted[i])

lion dance troupes international performers popular disneyland cartoon characters and colourful floats turn the parade into a big outdoor party
lion dance troke's international performar's popular disneyland cartoon characters and colourful floats turn the parade into a big outdoor party
for instance teachers could use crossword puzzles to help pupils learn both the spelling and the meaning of a word
for instance teachers could use crossword puzzles to help pupils learn both the spelling and the meaning of award
forum users claimed the decal has been removed from the windscreen
forum users claim the decal has been removed from the windscreen
and when guilt is proven the punishment for such serious crimes must be commensurate with the gravity of the crime
and when guilt is proven the punishment for such serious crimes must be commentared with the gravity of the crime
he became very fit ever since his break up last year
he became very fit ever since his breakup last year
a pinch of salt 

In [85]:
def enable_bn_se(m):
    if type(m) == nn.BatchNorm1d:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

    if 'SqueezeExcite' in type(m).__name__:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

quartznet.encoder.freeze()
quartznet.encoder.apply(enable_bn_se)
print("Model encoder has been frozen, and batch normalization has been unfrozen")

Model encoder has been frozen, and batch normalization has been unfrozen


In [86]:
quartznet.decoder.freeze()
quartznet.decoder.apply(enable_bn_se)
print("Model decoder has been frozen, and batch normalization has been unfrozen")

Model decoder has been frozen, and batch normalization has been unfrozen


In [87]:
## Config Information 
try:
    from ruamel.yaml import YAML
except ModuleNotFoundError:
    from ruamel_yaml import YAML
config_path = './configs/config.yaml'

yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
print(params)

{'name': 'QuartzNet15x5', 'sample_rate': 16000, 'repeat': 1, 'dropout': 0.0, 'separable': True, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'model': {'train_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'batch_size': 32, 'trim_silence': True, 'max_duration': 16.7, 'shuffle': True, 'is_tarred': False, 'tarred_audio_filepaths': None, 'shuffle_n': 2048, 'bucketing_strategy': 'synced_randomized', 'bucketing_batch_size': None}, 'validation_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'batch_size': 32, 'shuffle': False}, 'preprocessor': {'_target_': 'nemo.collections

In [43]:
## Importing libraries 
import json
import zipfile
import os 
import librosa
import torch
import torch.nn as nn
import copy
import pytorch_lightning as pl
import time 
import torchaudio
import noisereduce as nr
from scipy.io import wavfile
import numpy as np
import soundfile as sf
from pathlib import Path
from nemo.collections.tts.torch.helpers import BetaBinomialInterpolator
from nemo.collections.tts.models import HifiGanModel
from nemo.collections.tts.models import FastPitchModel
import IPython.display as ipd
from pydub import AudioSegment

In [37]:
def infer(spec_gen_model, vocoder_model, str_input):
    """
    Synthesizes spectrogram and audio from a text string given a spectrogram synthesis and vocoder model.
    
    Args:
        spec_gen_model: Spectrogram generator model (FastPitch in our case)
        vocoder_model: Vocoder model (HiFiGAN in our case)
        str_input: Text input for the synthesis
        speaker: Speaker ID
    
    Returns:
        spectrogram and waveform of the synthesized audio.
    """
    with torch.no_grad():
        parsed = spec_gen_model.parse(str_input)
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, speaker=None)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

def get_best_ckpt_from_last_run(
        base_dir, 
        new_speaker_id, 
        mixing_enabled, 
        model_name="FastPitch"
    ):    
    mixing = "no_mixing" if not mixing_enabled else "mixing"
    
    d = f"S{new_speaker_id}_{mixing}"
    
    exp_dirs = list([i for i in (Path(base_dir) / d / model_name).iterdir() if i.is_dir()])
    last_exp_dir = sorted(exp_dirs)[-1]
    
    last_checkpoint_dir = last_exp_dir / "checkpoints"
    
    last_ckpt = list(last_checkpoint_dir.glob('*-last.ckpt'))

    if len(last_ckpt) == 0:
        raise ValueError(f"There is no last checkpoint in {last_checkpoint_dir}.")
    
    return str(last_ckpt[0])

In [45]:
text_dir = '/home/dgxuser/Desktop/kailin/script_kl.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/vocab/train/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_train_manifest.json'

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

n = 0
speakers = [91, 124, 135, 148, 173, 178, 194, 208, 289, 245]

with open(manifest_path, 'w') as fout: 
    for speaker_id in speakers: 
        mixing = False
        last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
        print(last_ckpt)
        spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
        spec_model.eval().cuda()

        with open(text_dir, "r") as f: 
            while True: 
              text = f.readline().strip()
              if text == '': 
                break 
              spec, audio = infer(spec_model, vocoder, text)
              audio = ipd.Audio(audio, rate=22050)
              audio = AudioSegment(audio.data)
              file_dir = audio_dir + str(n) + '.wav'
              audio.export(file_dir, format="wav")

              duration = librosa.core.get_duration(filename=file_dir)
              metadata = {
                "audio_filepath": file_dir,
                "duration": duration, 
                "text": text
              }
              json.dump(metadata, fout)
              fout.write('\n')   
              n = n + 1 

[NeMo I 2022-07-13 10:38:17 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-13 10:38:17 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-13 10:38:17 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-13 10:38:21 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-13 10:38:21 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-13 10:38:21 features:200] PADDING: 0


[NeMo W 2022-07-13 10:38:21 features:177] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-07-13 10:38:21 features:200] PADDING: 0
[NeMo I 2022-07-13 10:38:22 save_restore_connector:243] Model HifiGanModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
S91_no_mixing/FastPitch/2022-07-12_17-20-12/checkpoints/FastPitch--v_loss=1.1944-epoch=24-last.ckpt
[NeMo I 2022-07-13 10:38:27 tokenize_and_classify:87] Creating ClassifyFst grammars.


...

[NeMo I 2022-07-13 10:44:34 features:200] PADDING: 1
S289_no_mixing/FastPitch/2022-07-12_09-59-53/checkpoints/FastPitch--v_loss=0.7410-epoch=24-last.ckpt
[NeMo I 2022-07-13 10:44:45 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-13 10:45:17 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-13 10:45:17 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S289_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_

[NeMo I 2022-07-13 10:45:17 features:200] PADDING: 1
S245_no_mixing/FastPitch/2022-07-12_10-14-25/checkpoints/FastPitch--v_loss=0.9827-epoch=49-last.ckpt
[NeMo I 2022-07-13 10:45:28 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-13 10:46:01 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-13 10:46:01 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S245_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_

[NeMo I 2022-07-13 10:46:01 features:200] PADDING: 1


In [44]:
text_dir = '/home/dgxuser/Desktop/kailin/script_kl_val.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/vocab/val/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_val_manifest.json'

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

n = 0
speakers = [91, 124, 135, 148, 173, 178, 194, 208, 289, 245]

with open(manifest_path, 'w') as fout: 
    for speaker_id in speakers: 
        mixing = False
        last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
        print(last_ckpt)
        spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
        spec_model.eval().cuda()

        with open(text_dir, "r") as f: 
            while True: 
              text = f.readline().strip()
              if text == '': 
                break 
              spec, audio = infer(spec_model, vocoder, text)
              audio = ipd.Audio(audio, rate=22050)
              audio = AudioSegment(audio.data)
              file_dir = audio_dir + str(n) + '.wav'
              audio.export(file_dir, format="wav")

              duration = librosa.core.get_duration(filename=file_dir)
              metadata = {
                "audio_filepath": file_dir,
                "duration": duration, 
                "text": text
              }
              json.dump(metadata, fout)
              fout.write('\n')   
              n = n + 1 

[NeMo I 2022-07-13 10:31:28 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-13 10:31:28 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-13 10:31:28 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-13 10:31:31 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-13 10:31:31 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-13 10:31:31 features:200] PADDING: 0


[NeMo W 2022-07-13 10:31:31 features:177] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-07-13 10:31:31 features:200] PADDING: 0
[NeMo I 2022-07-13 10:31:33 save_restore_connector:243] Model HifiGanModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
S91_no_mixing/FastPitch/2022-07-12_17-20-12/checkpoints/FastPitch--v_loss=1.1944-epoch=24-last.ckpt
[NeMo I 2022-07-13 10:31:38 tokenize_and_classify:87] Creating ClassifyFst grammars.


...

[NeMo I 2022-07-13 10:36:58 features:200] PADDING: 1
S289_no_mixing/FastPitch/2022-07-12_09-59-53/checkpoints/FastPitch--v_loss=0.7410-epoch=24-last.ckpt
[NeMo I 2022-07-13 10:37:04 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-13 10:37:36 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-13 10:37:36 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S289_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_

[NeMo I 2022-07-13 10:37:37 features:200] PADDING: 1
S245_no_mixing/FastPitch/2022-07-12_10-14-25/checkpoints/FastPitch--v_loss=0.9827-epoch=49-last.ckpt
[NeMo I 2022-07-13 10:37:43 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-13 10:38:16 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-13 10:38:16 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S245_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_

[NeMo I 2022-07-13 10:38:16 features:200] PADDING: 1


In [92]:
train_manifest_path = '/home/dgxuser/Desktop/kailin/TTS_train_manifest.json'
val_manifest_path = '/home/dgxuser/Desktop/kailin/TTS_val_manifest.json'

# add in manifest paths 
params['model']['train_ds']['manifest_filepath'] = train_manifest_path
params['model']['validation_ds']['manifest_filepath'] = val_manifest_path

# Point to the new validation data for fine-tuning
quartznet.setup_validation_data(val_data_config=params['model']['validation_ds'])

# Point to the data we'll use for fine-tuning as the training set
quartznet.setup_training_data(train_data_config=params['model']['train_ds'])

# Create a PyTorch Lightning trainer and call `fit` again.
trainer = pl.Trainer(max_epochs=3, accelerator='gpu', devices=1)
trainer.fit(quartznet)

[NeMo I 2022-07-13 11:28:44 collections:193] Dataset loaded with 50 files totalling 0.05 hours
[NeMo I 2022-07-13 11:28:44 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2022-07-13 11:28:44 collections:193] Dataset loaded with 520 files totalling 0.58 hours
[NeMo I 2022-07-13 11:28:44 collections:194] 0 files were filtered totalling 0.00 hours


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
[NeMo W 2022-07-13 11:28:44 modelPT:484] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2022-07-13 11:28:44 modelPT:585] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.01
        weight_decay: 0.001
    )


[NeMo W 2022-07-13 11:28:44 lr_scheduler:816] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 29.7 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
75.698    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [95]:
manifest_path = '/home/dgxuser/Desktop/kailin/manifest_kl.json'

transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [96]:
for i in range(len(transcript)): 
    if (transcript[i] != predicted[i]): 
        print(transcript[i])
        print(predicted[i])

but who can be automated
after foll came bhe autoor maycar
but to be fair to contractors he made sure they were paid on time every month
but to be fair too condractors he made sure they were pad on time every month o
if you happened to be any of these people
if you hapten to be any of thers piy car
a group photo will be taken after the session
a group photo will be taken after the sesson
also arrested for not remembering your dance moves
alsol resster from not remembering ou danance moves
in april last year the position changed
in april last year the posision changed
it will connect the northwestern and eastern regions of singapore to the city centre
it well come nopt the northwlestterd and eastern reagion of singapore to the city centre
need to service a couple of units but worried about the price
eed to slultthuce a couple of units but worried about the price
folks were understandably furious as well and most sided with the father
frolks were understandably furious as well and role s

In [97]:
error = wer(transcript, predicted)
print('Word Error Rate (Resampled): ' + str(error))

Word Error Rate (Resampled): 0.39335664335664333


In [83]:
error = cer(transcript, predicted)
print('Character Error Rate (Resampled): ' + str(error))

Character Error Rate (Resampled): 0.023123324396782843
